In [ ]:

import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl import workbook
import xlrd
import os
import numpy as np
import datetime as dt
pd.set_option('display.max_columns', None)

# Flag materials inside of tasks or other items 
def material_in_task_col(row):
    if row["Category"] == "Material" and row["Hierarchy #"] > 2:
        return 'Inside Task'
    else:
        return " "
# Flag materials without cost ID
def material_no_cost_ID(row):
    if row["Category"] == ("Material" or row["Category"] == "Task") and row["Job Cost ID"] == "":
        return 'No'
    else:
        return " "

# Flag LS items
def is_ls_um(row):
    if row["UM"] == "LS":
        return 'Yes'
    else:
        return " "

# check if unit cost matches default rate from DBs  
def UC_match(row):
    if row["Unit Direct Cost"] != row["Default Unit Cost"]:
        return 'No'
    else:
        return " "

# file pathes to b2w report "Cost Detail With Job Cost ID" and to where it should get saved once modified 
new_file_Path = r"C:\Users\will.davis\Desktop\Estimate reports\Cost Detail With Job Cost ID1.xlsx"
og_path = r"C:\Users\will.davis\Desktop\Estimate reports\Cost Detail With Job Cost ID.xlsx"

# list of lists containg reference file paths, the column to search, and the category that should be assigned 
#    [pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_pay_item_db.xlsx"), "name", "Pay Item"],
estimate_reference_paths = [
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_task_db.xlsx"), "Task Name", "Task"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_materials_db.xlsx"),"Material Type", "Material"],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_misc_db.xlsx"),"Name", "Misc."],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_labor_db.xlsx"), "Labor Type", "Labor", 'Unit Reg. Cost'],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_equipment_db.xlsx"),"Equipment Name", "Equipment", 'Unit Cost'],
[pd.read_excel(r"C:\Users\will.davis\Desktop\Estimate reports\b2w reference data\Estimate_crew_db.xlsx"), "Crew Name", "Crew"]]

# Read excel workbook of Cost Detail With Job Cost ID
df1 = pd.read_excel(og_path, skiprows=12)

# Remove blank columns/rename columns/dropna
df2 = df1.drop(df1.columns[[2,6,7,8,9,10,11,13,14,16]], axis=1)
df2.rename(columns={"Unnamed: 1":"Pay Item", "Unnamed: 3":"Task/Componenet", "Unnamed: 4":"Crew/Componenet", "Unnamed: 5":"Labor/Equipment",
                          "Direct Cost":"Unit Direct Cost","Direct Cost.1":"Total Direct Cost"},inplace=True)
df2.dropna(subset="Unit Direct Cost", inplace=True)
df3 = df2[df2["Unit Direct Cost"].str.contains("$")]
df3 = df3.dropna(subset="UM")
df3 = df3[df3['Job Cost ID']!= "Job Cost ID"]

# fill blank pay items with item above for hiararchy
df3['Pay Item'] = df3['Pay Item'].ffill()
df3[["Category","Hierarchy #", "Hierarchy Structure"]] = "Custom","",""
# create blank columns for different flags
df3[["UM Cost ID","UM Used", "Material Inside Task","Has Cost ID", "Is LS","Most Expensive Code", "Default Unit Cost",'Correct Unit Cost',"Recomended Cost ID","lab/equip"]] = ""
# establish categories and hierarchy 
for index, row1 in df3.iterrows():
    hierarchy = 0 
    for col1 in range(1,5):
        hierarchy += 1
        if pd.notna(row1.iloc[col1]):
            no_bracket_value = str(row1.iloc[col1]).split(" [")[0]
            df3.at[index,"Hierarchy #"] = hierarchy
            # print(no_bracket_value)
            for cat in estimate_reference_paths:
                if cat[2] == "Crew":
                    no_bracket_value = no_bracket_value.split(" (")[0]
                if no_bracket_value in list(cat[0][cat[1]]):
                    df3.at[index,"Category"] = cat[2]
                    # compare labor and equpment rates 
                    if cat[2] in  ["Equipment","Labor"]:
                        df3.at[index,"lab/equip"] = no_bracket_value
                        unit_cost = cat[0].loc[cat[0][cat[1]] == no_bracket_value, cat[3]]
                        just_cost = unit_cost.iloc[0]
                        df3.at[index,"Default Unit Cost"] = float(just_cost)             
df3.loc[df3['Hierarchy #'] == 1, "Category"] = "Pay Item"

# flag items with same cost ID but different Ums that are not materials or misc
# flag all, will adjust for subbed and materials thot don't matter
cost_id_list = list(df3["Job Cost ID"].unique())
for cost_id in cost_id_list:
    df_filtered = df3[df3["Job Cost ID"] == cost_id]
    UM_list = list(df_filtered['UM'].unique())
    UM_str = " ,".join(UM_list)
    if len(UM_list) != 1:
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Cost ID"] = "Different"
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Used"] = UM_str
    else:
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Cost ID"] = "Same"
        df3.loc[df3['Job Cost ID'] == cost_id, "UM Used"] = UM_str

# flag material in task, LS Items, missing cost ID
df3 = df3.fillna("")
df3["Material Inside Task"] = df3.apply(material_in_task_col, axis=1)
df3["Has Cost ID"] = df3.apply(material_no_cost_ID, axis=1)
df3["Is LS"] = df3.apply(is_ls_um, axis=1)

# madify cost str and convert to float 
df3['Unit Direct Cost'] = df3['Unit Direct Cost'].str.replace('$', '', regex=False)
df3['Unit Direct Cost'] = df3['Unit Direct Cost'].str.replace(',', '', regex=False)
df3['Unit Direct Cost'] = pd.to_numeric(df3['Unit Direct Cost'], errors='coerce')
df3['Total Direct Cost'] = df3['Total Direct Cost'].str.replace('$', '', regex=False)
df3['Total Direct Cost'] = df3['Total Direct Cost'].str.replace(',', '', regex=False)
df3['Total Direct Cost'] = pd.to_numeric(df3['Total Direct Cost'], errors='coerce')
# check unit price for labor and equipment 
df3['Correct Unit Cost'] = df3.apply(UC_match, axis=1)

# flag cost ID(first 3 digits) that has the greatest value in its pay item 
pay_item_list = list(df3["Pay Item"].unique())
for pay_item in pay_item_list:
    df_filtered1 = df3[df3["Pay Item"] == pay_item]
    max_value_index = df_filtered1['Total Direct Cost'].idxmax()
    # print(max_value_index)
    max_value_code = df_filtered1.loc[max_value_index, "Job Cost ID"]
    df3.loc[df3['Pay Item'] == pay_item, "Most Expensive Code"] = max_value_code



C:\Users\will.davis\AppData\Local\Temp\ipykernel_12272\2302285758.py:106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df3 = df3.fillna("")


In [ ]:
#to do
# flag potentially wrong Cost ID when compared to Cost Id reference sheet from Jill 



# some what complete
# flag items with same cost ID but different Ums that are not materials or misc  ## DONE
# Flag materials inside tasks # i think this is correct 
# flag materials that do not have Cost IDs
# flag all lump sum items 
# flag cost ID(first 3 digits) that has the greatest value in its pay item 
# checking quipment and labor rates, flag old rates if there is a delta between current 
# flag potentially custom Items, they may need complete restructure # by default items are custom and changed if it matches a db item 

# not pressing 
# flag items that have UM of Day/Month/LS that a foreman can charge to back burner 
# flag pay items that have taskes with different work types
# flag no cost code on little blue truck # little blue trucks will come in a s custome which need to be checked 



In [176]:

df3.head(5)

,Description,Pay Item,Task/Componenet,Crew/Componenet,Labor/Equipment,Job Cost ID,Quantity,UM,Unit Direct Cost,Total Direct Cost,Category,Hierarchy #,Hierarchy Structure,UM Cost ID,UM Used,Material Inside Task,Has Cost ID,Is LS,Most Expensive Code,Incorrect Unit Cost,Default Unit Cost,Recomended Cost ID,lab/equip,Correct Unit Cost
0,,AD - Area Drain / Inlet,,,,,0.00,EACH,0.00,0.0,Pay Item,1,,,,,,,,,,,,No
1,,AD - Area Drain / Inlet,080.370. Area Inlet,,,080.370.,0.00,EACH,0.00,0.0,Task,2,,Same,EACH,,,,,,,,,No
2,,AD - Area Drain / Inlet,,"Storm Sewer Manhole / Inlets (2.00 PH/EACH, 0.00",,,0.00,EACH,0.00,0.0,Crew,3,,,,,,,,,,,,No
4,,AD - Area Drain / Inlet,,,Excavator Cat 349/Kom 400-490,,0.00,HR,98.74,0.0,Equipment,4,,,,,,,,,98.74,,Excavator Cat 349/Kom 400-490,
5,,AD - Area Drain / Inlet,,,Loader Cat 938 - 950,,0.00,HR,50.67,0.0,Equipment,4,,,,,,,,,50.67,,Loader Cat 938 - 950,


In [ ]:

# need a method to identify crews as crews 

                # print(row1.iloc[col1])
                # if cat[2] == "Crew":
                #     no_bracket_value = no_bracket_value.split("(")[0]